#Source Code Tugas Proyek Akhir  
##Kelompok 5 JST - TIF B  
Aldi Fianda Putra - 195150201111034
                                                
Alif Ayodya - 195150207111006       

Nata Nirwasita Rasendriya	- 195150200111001

Hasyir Daffa Ibrahim - 195150207111039

# **A. Retrieve Dataset**

In [ ]:
#Install dependency
!pip install opendatasets
#!pip install neupy

#Import library
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import opendatasets as od
import csv
import numpy as np
import pandas as pd

In [ ]:
#Create kaggle.json file that contain your credential
!echo "{\"username\":\"USERNAME\",\"key\":\"KEY\"}" > kaggle.json

In [ ]:
#Checking kaggle.json file
!cat kaggle.json

In [ ]:
#Downloading dataset
od.download("https://www.kaggle.com/prathamtripathi/drug-classification")

print()

#Checking downloaded file
print("Daftar file")
!ls

100%|██████████| 1.68k/1.68k [00:00<00:00, 602kB/s]



Daftar file
drug-classification  kaggle.json  sample_data


In [ ]:
#Move csv file to main directory (cause we are using google colab)
! cp drug-classification/drug200.csv drug200.csv

#Checking csv file
print("Daftar file:")
!ls

Daftar file:
drug200.csv  drug-classification  kaggle.json  sample_data


# **B. Preprocessing Data**

## **Ekstraksi Data**

In [ ]:
#Extract data
def extract(file):
    return pd.read_csv(file)

In [ ]:
#Calling extract function
df_drug = extract("drug200.csv")

In [ ]:
#Checking dataframe
df_drug

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


## **Transformasi Data**

In [ ]:
#Checking null value
print("Null Value pada kolom Age:",df_drug['Age'].isnull().values.any())
print("Null Value pada kolom Sex:",df_drug['Sex'].isnull().values.any())
print("Null Value pada kolom BP:",df_drug['BP'].isnull().values.any())
print("Null Value pada kolom Cholesterol:",df_drug['Cholesterol'].isnull().values.any())
print("Null Value pada kolom Na_to_K:",df_drug['Na_to_K'].isnull().values.any())
print("Null Value pada kolom Drug:",df_drug['Drug'].isnull().values.any())

Null Value pada kolom Age: False
Null Value pada kolom Sex: False
Null Value pada kolom BP: False
Null Value pada kolom Cholesterol: False
Null Value pada kolom Na_to_K: False
Null Value pada kolom Drug: False


In [ ]:
#Checking unique value

print("Unique Value pada kolom Sex:",df_drug['Sex'].unique())
print("Unique Value pada kolom BP:",df_drug['BP'].unique())
print("Unique Value pada kolom Cholesterol:",df_drug['Cholesterol'].unique())
print("Unique Value pada kolom Drug:",df_drug['Drug'].unique())

Unique Value pada kolom Sex: ['F' 'M']
Unique Value pada kolom BP: ['HIGH' 'LOW' 'NORMAL']
Unique Value pada kolom Cholesterol: ['HIGH' 'NORMAL']
Unique Value pada kolom Drug: ['DrugY' 'drugC' 'drugX' 'drugA' 'drugB']


In [ ]:
#Transform categorical to numerical

df_drug['Sex'].replace({"F": 0, "M": 1}, inplace=True)

df_drug['BP'].replace({"HIGH": 2, "NORMAL":1, "LOW":0}, inplace=True)

df_drug['Cholesterol'].replace({"HIGH": 2, "NORMAL":1, "LOW":0}, inplace=True)

df_drug['Drug'].replace({"drugA": 0, "drugB": 1, "drugC": 2, "drugX": 3, "DrugY": 4}, inplace=True)

In [ ]:
#Checking value again after transform

print("Unique Value pada kolom Sex:",df_drug['Sex'].unique())
print("Unique Value pada kolom BP:",df_drug['BP'].unique())
print("Unique Value pada kolom Cholesterol:",df_drug['Cholesterol'].unique())
print("Unique Value pada kolom Drug:",df_drug['Drug'].unique())

Unique Value pada kolom Sex: [0 1]
Unique Value pada kolom BP: [2 0 1]
Unique Value pada kolom Cholesterol: [2 1]
Unique Value pada kolom Drug: [4 2 3 0 1]


In [ ]:
#Checking dataset value
df_drug

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,2,2,25.355,4
1,47,1,0,2,13.093,2
2,47,1,0,2,10.114,2
3,28,0,1,2,7.798,3
4,61,0,0,2,18.043,4
...,...,...,...,...,...,...
195,56,0,0,2,11.567,2
196,16,1,0,2,12.006,2
197,52,1,1,2,9.894,3
198,23,1,1,1,14.020,3


In [ ]:
#Exporting file
from google.colab import files


with pd.ExcelWriter("data_transform.xlsx") as writer:
    df_drug.to_excel(writer)

#Xlsx file for manualization
files.download('data_transform.xlsx') 

# **C. Learning Vector Quantization**

## **Fungsi Traning LVQ**

In [ ]:
#import Dependencies
import numpy as np

In [ ]:
def lvq_fit(train, target, lrate, b, max_epoch):
    label, train_idx = np.unique(target, return_index=True) 
    weight = train[train_idx].astype(np.float64)
    train = np.array([e for i, e in enumerate(zip(train, target)) if i not in train_idx])
    train, target = train[:, 0], train[:, 1] 
    epoch = 0

    while epoch < max_epoch:
      for i, x in enumerate(train):
          distance = [sum((w - x) ** 2) for w in weight] 
          min = np.argmin(distance)
          sign = 1 if target[i] == label[min] else -1 
          weight[min] += sign * lrate * (x - weight[min])

      lrate *= b 
      epoch += 1


    return weight, label

## **Fungsi Testing LVQ**

In [ ]:
def lvq_predict(X, model):
  center, label = model
  Y = []
  for x in X:
    d = [sum((c - x) ** 2) for c in center]
    Y.append(label[np.argmin(d)])
  return Y  

## **Fungsi Hitung Akurasi**

In [ ]:
def calc_accuracy(a, b):
  s = [1 if a[i] == b[i] else 0 for i in range(len(a))]
  return sum(s) / len(a)

In [ ]:
from random import uniform
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs, make_classification

array = df_drug.values
X = array[:,0:5]
y = array[:,5]
#X = X.astype(int)
y = y.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = lvq_fit(X_train, y_train, lrate=.3, b=.5, max_epoch=500)
output = lvq_predict(X_test, model)
accuracy = calc_accuracy(output, y_test)
colors = 'rgbcmyk'

print("Kelas asli:")
print(y_test)

print("Kelas hasil training:")
print(output)
print('Accuracy:', accuracy)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


Kelas asli:
[4 4 4 4 3 3 3 4 3 3 1 1 1 3 1 3 4 2 3 0 0 4 0 3 3 2 3 4 2 4 4 4 3 4 3 0 4
 4 1 4]
Kelas hasil training:
[4, 4, 4, 4, 4, 2, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 0, 4, 4, 4, 4, 4, 3, 0, 4, 3, 4, 4, 4, 3, 4, 3, 0, 4, 4, 4, 4]
Accuracy: 0.6
